In [1]:
from shapely.geometry import box
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from shapely.ops import unary_union
from fiona.crs import from_epsg
import os
import numpy as np

In [2]:
from geopy.distance import geodesic

def meters_to_degrees(meters, latitude):
    # Calculate the distance per degree at the specified latitude
    meters_per_degree = geodesic((latitude, 0), (latitude + 1, 0)).meters

    # Convert meters to degrees
    degrees = meters / meters_per_degree

    return degrees

In [3]:

def generate_bounding_boxes(tiff_path, box_size):
    """
    Generate fixed-sized square bounding boxes inside a given TIFF file.

    Parameters:
    - tiff_path: Path to the TIFF file
    - box_size: Size of the square bounding boxes

    Returns:
    - List of Shapely Polygon objects representing the bounding boxes
    """
    # Open the TIFF file and get its bounds
    bounding_boxes = []
    imgs = []
    
    with rasterio.open(tiff_path) as src:
        
        bounds = box(*src.bounds)
        
        avg_lat = (bounds.bounds[1] + bounds.bounds[2]) / 2
        box_size = meters_to_degrees(box_size, avg_lat)
        # Calculate the number of boxes in the x and y directions
        num_boxes_x = int((bounds.bounds[2] - bounds.bounds[0]) // box_size)
        num_boxes_y = int((bounds.bounds[3] - bounds.bounds[1]) // box_size)
        print(num_boxes_x, num_boxes_y)
        # Generate the bounding boxes
        cnt = 0
        cnt2 = 0
        for i in range(num_boxes_x):
            for j in range(num_boxes_y):
                # Calculate the coordinates of the box
                min_x = bounds.bounds[0] + i * box_size
                min_y = bounds.bounds[1] + j * box_size
                max_x = min_x + box_size
                max_y = min_y + box_size

                # Create a Shapely box
                box_geometry = box(min_x, min_y, max_x, max_y)

                # Check if the box is completely within the TIFF bounds
                
                if bounds.contains(box_geometry):                    
                    cnt+=1
                    window = src.window(min_x, min_y, max_x, max_y)
                    image_array = src.read(window=window)
                    pixels = image_array.shape[0] * image_array.shape[1]
                    if np.count_nonzero(image_array) >= pixels * 0.5:  
                        cnt2 += 1
                        imgs.append(image_array)                    
                        bounding_boxes.append(box_geometry)
        print(cnt, cnt2)
                    

    return bounding_boxes, imgs




In [4]:
tiff_path = '../data/tif_labels/'  
box_size = 5120

In [5]:
tif_labels_path = [os.path.join(tiff_path, file) for file in os.listdir(tiff_path) if file.endswith(".tif")]

In [6]:
tif_labels_path

['../data/tif_labels/TropicalTreeCover_NorthernQLD-0000131072-0000196608.tif']

In [12]:
target_crs = from_epsg(4326)
for i, path in enumerate(tif_labels_path):
    # Generate bounding boxes
    bounding_boxes, imgs = generate_bounding_boxes(path, box_size)

    # Save the bounding boxes to a GeoJSON file
    gdf = gpd.GeoDataFrame(geometry=bounding_boxes, crs=target_crs)
    gdf['height'] = (imgs[0].shape[1])
    gdf['width'] = (imgs[0].shape[2])
#     print(gdf)
    gdf.to_file('bounding_boxes{bid}.geojson'.format(bid = i), driver='GeoJSON')

131 74
9694 634


In [13]:
imgs[0].shape

(1, 523, 499)

In [14]:
gdf

,geometry,height,width
0,"POLYGON ((144.36207 -21.34429, 144.36207 -21.2...",523,499
1,"POLYGON ((144.36207 -21.29835, 144.36207 -21.2...",523,499
2,"POLYGON ((144.36207 -21.25240, 144.36207 -21.2...",523,499
3,"POLYGON ((144.36207 -21.20645, 144.36207 -21.1...",523,499
4,"POLYGON ((144.36207 -21.16051, 144.36207 -21.1...",523,499
...,...,...,...
629,"POLYGON ((145.64858 -20.97672, 145.64858 -20.9...",523,499
630,"POLYGON ((145.64858 -20.93077, 145.64858 -20.8...",523,499
631,"POLYGON ((145.64858 -20.88482, 145.64858 -20.8...",523,499
632,"POLYGON ((145.64858 -20.83888, 145.64858 -20.7...",523,499


In [15]:
for i in range(100):
    np.save("../data/labels/{iid}".format(iid = i), imgs[i])

In [29]:
mask = np.where(imgs[0] <= 35, 0, 1)

In [31]:
np.count_nonzero(mask)

185